# Assignment2
## Anomaly detection 데이터셋을 SVM을 이용해서 판별해보자
### 데이터 분포 : True : 0.17프로 나머지 다 False, Row약 28만행 < 데이터 트레인 시간이 오래걸립니다.(그래도 자고 일어나면 트레이닝 돼있기는 하는정도입니다.!!)
### 여러가지 방법을 사용해서 트레이닝 해보고 결과를 비교해주면됩니다!
### 시간이 없거나 좀 어렵다 하시는 분들은 멘토에게 물어보고 한 가지 방법으로만 구현해도 됩니다.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# 1. 데이터 Load 및 구조 확인

In [2]:
# Anomaly detection(사기감지 데이터) 로드
data = pd.read_csv('anomaly-detection/creditcard.csv')
print(data.columns)

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')


In [3]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
data.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284806.000000,284806.000000,2.848060e+05,284806.000000,284806.000000,2.848060e+05,284806.000000,284806.000000,284806.000000,284806.000000,...,2.848060e+05,284806.000000,284806.000000,2.848060e+05,284806.000000,284806.000000,2.848060e+05,2.848060e+05,284806.000000,284806.000000
mean,94813.585781,0.000002,6.661837e-07,-0.000002,0.000002,4.405007e-08,0.000002,-0.000006,0.000001,-0.000002,...,-9.166149e-07,-0.000002,-0.000001,-3.088756e-08,0.000002,0.000003,8.483873e-09,-4.792707e-08,88.349168,0.001727
std,47488.004530,1.958699,1.651311e+00,1.516257,1.415871,1.380249e+00,1.332273,1.237092,1.194355,1.098634,...,7.345251e-01,0.725702,0.624461,6.056481e-01,0.521278,0.482225,4.036332e-01,3.300838e-01,250.120432,0.041527
min,0.000000,-56.407510,-7.271573e+01,-48.325589,-5.683171,-1.137433e+02,-26.160506,-43.557242,-73.216718,-13.434066,...,-3.483038e+01,-10.933144,-44.807735,-2.836627e+00,-10.295397,-2.604551,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.250000,-0.920374,-5.985522e-01,-0.890368,-0.848642,-6.915995e-01,-0.768296,-0.554080,-0.208628,-0.643098,...,-2.283974e-01,-0.542351,-0.161846,-3.545895e-01,-0.317142,-0.326979,-7.083961e-02,-5.295995e-02,5.600000,0.000000
50%,84691.500000,0.018109,6.549621e-02,0.179846,-0.019845,-5.433621e-02,-0.274186,0.040097,0.022358,-0.051429,...,-2.945020e-02,0.006781,-0.011196,4.097671e-02,0.016596,-0.052134,1.342244e-03,1.124381e-02,22.000000,0.000000
75%,139320.000000,1.315645,8.037257e-01,1.027198,0.743348,6.119267e-01,0.398567,0.570426,0.327346,0.597140,...,1.863701e-01,0.528548,0.147641,4.395270e-01,0.350716,0.240955,9.104579e-02,7.828043e-02,77.160000,0.000000
max,172788.000000,2.454930,2.205773e+01,9.382558,16.875344,3.480167e+01,73.301626,120.589494,20.007208,15.594995,...,2.720284e+01,10.503090,22.528412,4.584549e+00,7.519589,3.517346,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [5]:
data.isnull().sum()
#모든 변수에 대해 결측치는 없다.

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [6]:
#데이터를 feature와 target으로 분리
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [7]:
X.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99


In [8]:
y

0         0
1         0
2         0
3         0
4         0
         ..
284801    0
284802    0
284803    0
284804    0
284805    0
Name: Class, Length: 284806, dtype: int64

In [9]:
y.value_counts()
#굉장히 비대칭적인 데이터이다.

0    284314
1       492
Name: Class, dtype: int64

# 2. Scailing

In [10]:
#X를 스케일링하여 업데이트
scaler = StandardScaler() #정규화를 하자!
X = scaler.fit_transform(X) # 스케일링을 해주자!

In [11]:
feature = ['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount']

In [12]:
X_df = pd.DataFrame(X, columns = feature)
X_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,-1.996583,-0.694242,-0.044075,1.672773,0.973363,-0.245116,0.347066,0.193684,0.082636,0.331129,...,0.326119,-0.024922,0.382857,-0.176909,0.110507,0.246582,-0.392178,0.330891,-0.063781,0.244966
1,-1.996583,0.608494,0.161175,0.109799,0.316521,0.043483,-0.061822,-0.063696,0.071252,-0.232492,...,-0.089609,-0.307375,-0.880073,0.162203,-0.561130,0.320691,0.261064,-0.022256,0.044608,-0.342472
2,-1.996562,-0.693500,-0.811577,1.169469,0.268230,-0.364571,1.351450,0.639781,0.207371,-1.378672,...,0.680976,0.337632,1.063361,1.456320,-1.138090,-0.628540,-0.288454,-0.137137,-0.181020,1.160686
3,-1.996562,-0.493325,-0.112170,1.182516,-0.609727,-0.007469,0.936147,0.192075,0.316016,-1.262500,...,-0.269853,-0.147442,0.007270,-0.304774,-1.941024,1.241900,-0.460225,0.155396,0.186188,0.140536
4,-1.996541,-0.591330,0.531540,1.021412,0.284654,-0.295015,0.071997,0.479307,-0.226511,0.744327,...,0.529940,-0.012838,1.100014,-0.220121,0.233250,-0.395205,1.041609,0.543619,0.651815,-0.073401


# 3. Imbalanced data problem
현재 데이터는 결제가 사기인지 아닌 지 판별하는 데이터로, 굉장히 imbalance한 구조로 되어 있다. 사기가 일어나는 경우가 그렇지 않은 경우보다 현저히 적기 때문!  이러한 데이터를 가지고 그대로 모델에 적합시키게 되면 모델은 자연스럽게 우세한 클래스, 즉 사기가 일어나지 않는 경우를 더 많이 학습하고 이로 인해 새로운 데이터가 들어와도 사기가 아니라고 분류할 가능성이 높아진다.  
하지만 사기인데 사기가 아니라고 하는 경우는 굉장히 치명적! 때문에 이 문제를 해결해야 한다.

## 3-1. UnderSampling
[이곳](https://blog.naver.com/PostView.nhn?blogId=wjddudwo209&logNo=221448123026&categoryNo=0&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView)을 참고하였습니다.

In [13]:
#데이터 셋 분류
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

In [14]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=0)
rus.fit(X_train, y_train)
X_rus, y_rus = rus.fit_resample(X_train, y_train)

두 클래스 데이터가 50대 50씩 들어가도록 random under sampling.   
데이터는 급격하게 줄어들지만...
그래도 한쪽에 치우쳐 학습되는 현상을 방지할 수 있다.

In [15]:
#언더 샘플링한 데이터의 차원 확인
print(X_rus.shape)
print(y_rus.shape)

(788, 30)
(788,)


In [16]:
#동일한 비율로 샘플링 되어 있다.
y_rus.value_counts()

1    394
0    394
Name: Class, dtype: int64

In [17]:
from sklearn import metrics

#Linear로 학습시켜서 결과 확인해보기
svc=SVC(kernel='linear') 
svc.fit(X_rus,y_rus)  #샘플링한 데이터를 fit
y_pred=svc.predict(X_test) #이를 바탕으로 예측값 생성
print('Accuracy Score:', metrics.accuracy_score(y_test,y_pred)) #Accuracy 계산

Accuracy Score: 0.9666268740563885


In [18]:
#RBF커널로 학습시켜서 결과 확인해보기
svc_rbf=SVC(kernel='rbf', gamma = 1)
svc_rbf.fit(X_rus,y_rus) #샘플링한 데이터를 fit
y_pred=svc_rbf.predict(X_test) #이를 바탕으로 예측값 생성
print('Accuracy Score:',metrics.accuracy_score(y_test,y_pred)) #Accuracy 계산

Accuracy Score: 0.9965591095818265


하이퍼파라미터 C의 디폴트는 1입니다.   
값이 이렇게 좋을 수가.. gamma값은 건드리지 않아도 되겠군요^^

In [19]:
# 교차검증 X 하이퍼파라미터 튜닝으로 모델 성능 높이기
from tqdm import tqdm
from sklearn.model_selection import cross_validate

#하이퍼 파라미터 C값 지정
C_range=list(np.arange(1,80,1)) # C값은 1부터 80까지 1간격
acc_score = [] # 결과를 보관할 list

#교차검증 (Cross validation)
for c in tqdm(C_range): #결과를 실시간으로 확인할 수 있게 지정하고
    svc = SVC(kernel='rbf', C=c, gamma = 'scale') #gamma에 디폴트 값을 scale로 해주기
    scores = cross_validate(svc, X_rus, y_rus, cv=5, scoring='accuracy') # 점수를 리스트로 관리해줍니다.
    score = np.mean(scores['test_score']) #5개 test score 값의 평균
    acc_score.append(score) #이를 acc_score에 넣음.
    
print("Best test score : " , np.max(acc_score)) #제일 좋은 결과
print("Best parameter : ", C_range[np.argmax(acc_score)]) #제일 좋은 결과가 나온 파라미터는 52였음.

100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:03<00:00, 23.22it/s]


Best test score :  0.9492461501249698
Best parameter :  52


Resampling 한 데이터 셋이 너무 작아 교차검증 하는 것이 별 의미가 없을 지도 모르나 일단 해보았습니다. 베스트 파라미터라고 나온 C 값이 52인 것이 좀 당혹스럽네요. 디폴트 값이 1인데 원래 이렇게 커질 수도 있나요?

## 3-2. OverSampling
randomoversampler와 SMOTE를 사용해보았습니다.  
하지만 데이터 규모에 의한 현실적 문제에 봉착하여 모델 적합은 하지 못했습니다.
[이곳](https://datascienceschool.net/view-notebook/c1a8dad913f74811ae8eef5d3bedc0c3/)을 참조하였습니다.

In [20]:
from imblearn.over_sampling import RandomOverSampler

#Random over sampling 
ros = RandomOverSampler(random_state=0)
ros.fit(X_train, y_train)
X_ros, y_ros = RandomOverSampler(random_state=0).fit_resample(X_train, y_train)

In [21]:
print(X_ros.shape)
print(y_ros.shape)

(454900, 30)
(454900,)


In [22]:
y_ros.value_counts()

1    227450
0    227450
Name: Class, dtype: int64

In [23]:
from imblearn.over_sampling import SMOTE

#SMOTE method 
X_smt, y_smt = SMOTE(random_state=4).fit_sample(X_train, y_train)

In [24]:
print(X_smt.shape)
print(y_smt.shape)

(454900, 30)
(454900,)


In [25]:
y_smt.value_counts()

1    227450
0    227450
Name: Class, dtype: int64

## 3-3. RandomSampling
그렇다면 데이터의 size도 보장되면서, class의 비율도 적절하게 조정해보는 것은 어떨까요?   오버샘플링 데이터를 바탕으로 랜덤 추출을 진행해보았습니다.

In [26]:
#Sampling을 편리하게 진행하기 위해 데이터 프레임 형태로 변환
X_df = pd.DataFrame(X_smt, columns = feature)
X_df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,1.625496,-0.271189,1.043019,-0.702155,0.709700,0.802851,-0.281031,0.840851,0.320818,-1.010843,...,0.057145,0.173856,0.633432,-0.295282,0.954714,0.058713,-0.880161,0.780446,0.641428,-0.237962
1,0.756075,-0.542849,0.089656,0.483015,-0.738627,0.568386,-0.741559,0.983819,-0.253577,0.139874,...,-0.297872,-0.155306,-0.016890,0.466154,-0.129937,0.307620,-0.386780,0.137790,0.229298,-0.133333
2,-1.088247,-0.479757,-0.180537,0.626280,-2.213826,1.586243,2.658142,-0.033958,0.123298,0.062517,...,0.969993,0.092051,1.326134,-0.641822,1.702455,0.586668,-0.576127,-0.504520,-1.665708,-0.253315
3,-0.872697,-1.078218,-0.296589,0.162068,-1.974703,1.209820,2.806403,-0.748857,-0.589793,-0.850611,...,-0.792213,1.876534,-0.615352,0.555132,1.642421,1.286488,-0.627030,1.473521,-0.135160,0.118307
4,-0.399672,0.577533,-0.039266,0.835213,1.245953,-0.615619,0.104344,-0.348231,0.101722,1.080721,...,-0.203351,-0.599668,-1.095093,0.085450,0.668950,0.988701,-1.120868,0.194615,0.098221,-0.327240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454895,-0.510450,-1.904546,0.228858,-2.482555,2.785735,-1.338483,-1.478531,-2.901133,1.349645,-2.020292,...,1.203555,1.089856,-0.181196,-0.651251,0.430703,-0.108335,-0.412429,2.961542,-0.219891,0.495702
454896,-1.124712,-7.677443,4.842082,-10.990725,6.092348,-9.799626,-2.140857,-12.139432,8.217497,-7.082356,...,-2.133943,3.928846,0.102325,-1.155067,1.038207,0.766075,0.200533,-4.839240,-3.449248,0.474488
454897,-0.560281,-6.735438,7.742932,-6.533631,2.345088,-3.478491,4.323488,-15.157250,-31.275054,-0.356387,...,-4.530985,37.034659,-12.246118,8.493113,-1.055788,0.504916,-0.225787,3.145352,2.845971,-0.349229
454898,-0.375576,-0.366675,0.358752,0.760866,-0.183290,-0.412024,-0.457643,0.568861,0.058190,-0.121391,...,-0.229264,0.182723,0.335810,-0.129802,0.674306,-0.415786,0.739046,-0.159396,0.021480,0.173808


In [27]:
#마찬가지로 class 값도 데이터 프레임 형태로 변환
y_df = pd.DataFrame(y_smt)
y_df

,Class
0,0
1,0
2,0
3,0
4,0
...,...
454895,1
454896,1
454897,1
454898,1


In [28]:
# 이 중 0.1퍼센트 랜덤으로 추출하고 X_sam에 저장
X_sam = X_df.sample(frac=0.03, random_state = 77)
X_sam

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
206578,-0.763112,0.606140,-0.037481,-0.230617,0.640829,0.448666,0.832637,-0.088288,0.249132,0.288812,...,-0.178712,-0.216785,-0.507816,-0.480895,-2.859675,1.562794,-0.459898,0.052293,-0.027962,-0.205298
411778,0.914162,0.433125,1.947796,-3.173676,3.207326,2.260368,-0.852296,0.806473,-1.385639,-1.953860,...,-0.461687,1.736501,-1.091565,0.397223,-1.925973,-1.921858,0.079350,0.460252,0.461072,-0.350584
265559,0.162823,-1.215321,2.539001,-4.452286,3.516069,-1.426440,-1.419715,-3.797031,1.028122,-1.987908,...,1.373456,1.623168,1.371619,-0.750975,-0.765352,0.573465,0.261153,1.554475,0.292870,-0.117004
353860,-0.825893,-0.625436,1.157818,-0.568683,1.924716,-1.398330,-0.035791,-1.092553,-0.432400,-1.068565,...,0.345742,1.614919,0.412544,0.040584,0.812406,-0.390218,-0.441008,1.379220,0.228787,0.335270
272705,-1.811961,-1.181335,3.421033,-4.945249,4.534496,0.005442,-2.368347,-2.405689,1.404653,-3.444457,...,0.539166,0.933928,-0.636649,-0.663563,-0.416721,2.020696,0.898162,1.093302,0.988000,-0.349229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294699,-1.837676,0.165605,2.271615,-3.767178,4.284303,1.175364,-1.885578,-0.635780,0.085623,-2.183592,...,0.411076,0.241489,-0.821962,-0.903807,-0.856968,2.760892,1.098624,1.302866,1.223026,-0.349229
268733,-1.392653,-14.779858,9.720874,-19.661149,4.566743,-15.259927,-3.669762,-15.911110,15.918699,-3.301200,...,2.221398,2.461505,-2.969340,-2.157711,0.285046,3.998805,-0.435801,3.189146,1.137015,0.046541
300817,-0.888146,-2.390902,1.784217,-3.475920,4.008211,-1.657777,-0.841647,-3.424657,0.518004,-1.357405,...,-0.627648,1.327411,0.354411,1.002879,0.315323,-1.298900,-0.289149,-3.541922,2.692569,-0.144069
163052,1.469267,-0.078847,0.673154,-0.315123,-0.580984,0.900233,-0.260866,0.848643,-0.170348,0.078206,...,0.289509,-0.511078,-1.029168,0.095458,0.410471,-0.733843,0.158901,0.079053,-0.289114,-0.349669


In [29]:
# 이 중 0.1 퍼센트를 랜덤으로 추출하고 y_sam에 저장
y_sam = y_df.sample(frac=0.03, random_state = 77)
y_sam

,Class
206578,0
411778,1
265559,1
353860,1
272705,1
...,...
294699,1
268733,1
300817,1
163052,0


#### 이제 샘플링한 데이터로 분석을 실시해봅시다!

In [30]:
#Linear로 학습시켜서 결과 확인해보기
svc=SVC(kernel='linear', verbose=True) 
svc.fit(X_sam,y_sam)  #샘플링한 데이터를 fit
y_pred=svc.predict(X_test) #이를 바탕으로 예측값 생성
print('Accuracy Score:', metrics.accuracy_score(y_test,y_pred)) #Accuracy 계산

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LibSVM]Accuracy Score: 0.9787051016467119


In [31]:
#RBF커널로 학습시켜서 결과 확인해보기
svc_rbf=SVC(kernel='rbf', gamma = 1)
svc_rbf.fit(X_sam,y_sam) #샘플링한 데이터를 fit
y_pred=svc_rbf.predict(X_test) #이를 바탕으로 예측값 생성
print('Accuracy Score:',metrics.accuracy_score(y_test,y_pred)) #Accuracy 계산

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy Score: 0.9979811102138267


Linear보다 RBF가 성능이 좋은데 딱 한 가지 경우
feature가 데이터의 개수보다 많은 경우에는 그렇지 않을 수 있다고 하네요   
허나 뭐 이런 일은 거의 없으니,,,, 

In [32]:
tuned_parameters = {
 'C': (np.arange(1,10,2)), 'kernel': ['rbf'], 'gamma': (np.arange(1,5,1))}

In [33]:
from sklearn.model_selection import GridSearchCV

svm_model= SVC()
model_svm = GridSearchCV(svm_model, tuned_parameters, cv=3 ,scoring='accuracy', verbose = 1)

In [34]:
model_svm.fit(X_sam, y_sam)
print(model_svm.best_score_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was pas

0.9381549058401113


고된 인내의 시간,,,,,,,,,,드디어 끝났군요,,,,,,

In [36]:
print(model_svm.best_params_) # 최고의 결과가 나온 경우는 C가 3일때, gamma가 1일 때, kernel이 rbf일 때 였습니다.

{'C': 3, 'gamma': 1, 'kernel': 'rbf'}


위에서 나온 0.99였을 때의 하이퍼 파라미터 값과 동일한데  
교차 검증에 의해서 평균값이 산출되는 바람에 0.93이 나왔나 봅니다 (맞지요?) 

### [결론] 
이 카드사기 데이터는 스케일과 두 번의 샘플링을 거쳐 C가 3일 때, gamma가 1일 때, kernel이 rbf일 때 가장 좋은 성능을 보였다! 

하지만 새로운 데이터 셋을 넣었을 때의 성능은 장담하지 못할 것 같다! --이유는 밑을 참고해주세요

### [아쉬웠던 점]
1. Sampling 할 때 ratio를 이용해 클래스의 비율을 다르게 추출해보고 싶었는데 ratio가 안 돌아가서 실패.. 결국 50대 50으로 추출할 수 밖에 없었네요

2. 마지막 Over random sampling의 결과를 또 sampling을 진행하여 분석을 수행했을 때 매 시행마다 accuracy값의 차이가 컸습니다. 아무래도 샘플링을 두 번 수행해서 일까요 ㅠ 0.4~0.99까지 값이 나왔고 random_state를 77로 고정한 현재 rbf로 돌렸을 때 0.99값입니다. 정녕 이래도 되는 것인지요....사기를 치는 듯한 느낌이 듭니다..

==> 이것도 일종의 overfitting 혹은 underfitting이라고 볼 수 있을까요? 
만약 실제 분석 상황이라면 이 상황을 어찌하면 좋을까요? 
